In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx 
import os
from scipy.stats import spearmanr,pearsonr
import sys
from math import log10
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
dir_ ='data'

In [11]:
NPX_data =  pd.read_excel(open(os.path.join(dir_,'20191510_Buckwalter_NPX.xlsx'),'rb'), sheet_name='NPX_Data', usecols = 'A:AUA',skiprows=5)
NPX_panel_type = pd.read_excel(open(os.path.join(dir_,'20191510_Buckwalter_NPX.xlsx'),'rb'), sheet_name='NPX_Data', usecols = 'A:AUA',nrows=4).transpose()
NPX_data['sel'] =NPX_data['OlinkID'].apply(lambda x:1 if str(x)[len(str(x))-2:]=='-0' else 0)
NPX_data = NPX_data[NPX_data['sel']==1]
NPX_data = NPX_data.drop('sel', axis=1)
NPX_data['OlinkID']= NPX_data['OlinkID'].apply(lambda x:x[:-2] if str(x)[len(str(x))-2:]=='-0' else x)
NPX_data = NPX_data.rename(columns={'OlinkID':'pid'})
NPX_data['pid'] = NPX_data['pid'].apply(lambda x:int(x))
ocols = [col for col in NPX_data.columns if 'OID' in col]
ocols.append('pid')
NPX_data = NPX_data[ocols]
NPX_data = NPX_data.dropna(axis=1)


In [12]:
patient_data = pd.read_excel(open(os.path.join(dir_,'StrokeCog_20200128_Restructured.xlsx'),'rb'), sheet_name='StrokeCog_RedCap_20200128_Olink', usecols = 'A:ML', skiprows=1)

In [13]:
patient_data['study_id'] = patient_data['study_id'].apply(lambda x: x[5:])
patient_data = patient_data.rename(columns={'study_id': 'pid'})
patient_data = patient_data.rename(columns={'Stroke_size':'size','demo_gender':'gender', 'nihss_total':'nihss', 'online_hhist_diabtype':'diabtype', 'online_hhist_hf':'CHF', 'online_hhist_valverepl':'hvalve', \
'online_hhist_pd':'Parkinsons', 'online_hhist_seiz':'seizure', 'online_hhist_dementia':'dementia','strokeToBattery_days.event_0_baseline':'timesince', \
'battery_age.event_0_baseline':'age', 'sis1_transformed_score.event_0_baseline':'sis1', 'sis2_transformed_score.event_0_baseline':'sis2', \
'sis3_transformed_score.event_0_baseline':'sis3', 'sis4_transformed_score.event_0_baseline':'sis4',\
'sis5_transformed_score.event_0_baseline':'sis5', 'sis6_transformed_score.event_0_baseline':'sis6', \
'sis7_transformed_score.event_0_baseline':'sis7', 'sis8_transformed_score.event_0_baseline':'sis8', 'facit_f_total.event_0_baseline':'facit'})
#patient_data['sis3'].head()
patient_data['pid'] = patient_data['pid'].apply(lambda x:int(x))

In [14]:
set1 = set(NPX_data['pid'].values.tolist())
set2 = set(patient_data['pid'].values.tolist())
intersection = set1.intersection(set2)
NPX_data= NPX_data[NPX_data['pid'].isin(intersection)]

In [15]:
patient_data = patient_data[patient_data['pid'].isin(intersection)]
featurenames = ["pid","size","gender","nihss","diabtype","CHF","hvalve",
      "Parkinsons","seizure","dementia","timesince","age","sis1","sis2","sis3","sis4",
      "sis5","sis6","sis7","sis8","facit"]
patient_data = patient_data[featurenames]

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from csv import writer 
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from scipy import stats

In [17]:
target = ['sis3'] #['facit']
add_fea = ['timesince','age']#,'size','gender','nihss',]#,
str_add_fea = 'pr +' + '+ '.join(add_fea)
print(str_add_fea)
y_data_features = []
y_data_features.extend(add_fea)
y_data_features.extend(target)
y_data_features.extend(['pid'])
y_data = patient_data.dropna(subset=target)
y_data = y_data[y_data_features] #'size','timesince','age',
y_data = y_data.dropna(subset=add_fea)

X = NPX_data

data = y_data.merge(X,how='inner', on='pid')

selected_features = [col for col in NPX_data if 'OID' in col]
print(len(selected_features))
selected_features.extend(add_fea)
pids = set(data['pid'].values.tolist())   

pr +timesince+ age
1011


In [18]:
model = LogisticRegression(fit_intercept=True)
tru_vals = []
predictions = []
fs_method = 'no filter' #Available feature selection methods: 'Kristy', 'MIR', 'RF', 'l2' 
x_shape = len(selected_features)
for i,p in enumerate(list(pids)):
    fs_x_tr =  data[data['pid']!=p][selected_features]
    fs_y_tr = data[data['pid']!=p][target]
    if fs_method == 'no filter':
        new_selected_features = selected_features
        print('no filter')
    else:
        new_selected_features = fs_func(fs_method, fs_x_tr, fs_y_tr, p)
    #print('new_selected_features:', new_selected_features)
    x_tr = data[data['pid']!=p][new_selected_features].values.reshape(-1, len(new_selected_features))
    y_tr = data[data['pid']!=p][target].values.reshape(-1, 1)
    x_te = data[data['pid']==p][new_selected_features].values.reshape(-1, len(new_selected_features))
    y_te = data[data['pid']==p][target].values.reshape(-1, 1)
    #print(y_te.flatten())
    tru_vals.extend(y_te.flatten())
    model.fit(x_tr, y_tr)
    y_pred = model.predict(x_te)
    predictions.extend(y_pred)
    #print('a')
List = [type(model).__name__, fs_method+str_add_fea, target,  spearmanr(predictions, tru_vals)[0], spearmanr(predictions, tru_vals)[1],  -log10(spearmanr(predictions, tru_vals)[1])]
file_name = 'FS/kristy_FS_%s_pvals_ipython2.csv'%target[0]
#file_name = 'FS/test.csv'
with open('Pvals/%s'%file_name, 'a') as f_object: 
    writer_object = writer(f_object) 
    writer_object.writerow(List) 
    f_object.close()


no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter
no filter


In [19]:
############################ FS 

In [20]:
def fs_func(fs_method, fs_x_tr, fs_y_tr, cv_n):
    X = fs_x_tr.values
    y = fs_y_tr.values
    if fs_method == 'Kristy':
        feat1 = get_kristy_prs()
        feat2 = add_fea
        k_feas = []
        k_feas.extend(feat1)
        k_feas.extend(feat2)
        return k_feas
    if fs_method == 'RF':
        model = LogisticRegression()
        rfe = RFE(model, 100)
        fit = rfe.fit(X, y)
        lst = zip(fit.ranking_, fs_x_tr.columns)
        lst = sorted(lst, key = lambda x:np.abs(x[0]))
    elif fs_method == 'l2':
        scaler = StandardScaler()
        scaler.fit(fs_x_tr)
        sel_ = SelectFromModel(LogisticRegression(penalty='l2'))
        sel_.fit(scaler.transform(fs_x_tr), fs_y_tr)
        ridge_selected_feat = fs_x_tr.columns[(sel_.get_support())]
        lst = zip(sel_.estimator_.coef_[0,:], fs_x_tr.columns)
        print(sel_.estimator_.coef_)
        lst = sorted(lst,  key = lambda x:np.abs(x[0]))
        ridge_df = pd.DataFrame(lst, columns =['feature','Score'])
        ridge_df ['cv']=cv_n
        write_to_file(fs_method, ridge_df)
        return ridge_selected_feat
    elif fs_method == 'MIR':
        test = SelectKBest(score_func=mutual_info_regression, k=100)
        fit = test.fit(X, y)
        lst = zip(fit.scores_, fs_x_tr.columns)
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    else:
        return fs_x_tr.columns
    fea_df = pd.DataFrame(lst, columns =['feature','Score'])
    fea_df ['cv']=cv_n
    write_to_file(fs_method, fea_df)
    feas = [name for coef, name in lst[:100]]
    #print(lst)
    return feas


In [21]:
def write_to_file(fs_method, df):
    #df.to_csv('FS/w-boostarpping/%s.csv'%(fs_method+str_add_fea), mode='a', header=False)
    return True